In [1]:
import ee
import geemap as emap

In [2]:
# Initialize ee
ee.Initialize()
ee.Authenticate()

Enter verification code: 4/1AX4XfWhLmegtoFh09gCQpgU40wz-04qrifN35sItha9xQenS9MYSfzlN1TY

Successfully saved authorization token.


In [3]:
Map=emap.Map()

Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

# <font color="blue"> 1. Create a image collection </font>

In [ ]:
# Create arbitrary constant images.
constant1 = ee.Image(1).rename("B1")
constant2 = ee.Image(2).rename("B1")
# Get info of image1
constant1.getInfo()

In [ ]:
# Create a collection by giving a list to the constructor.
collectionFromConstructor = ee.ImageCollection([constant1, constant2])
print('collectionFromConstructor: ', collectionFromConstructor.getInfo())

In [ ]:
# Create a collection with fromImages().
collectionFromImages = ee.ImageCollection.fromImages(
  [ee.Image(3.5), ee.Image(4)])
print('collectionFromImages: ', collectionFromImages.getInfo())

**We can merge two or more collection**

In [ ]:
# Merge two collections.
mergedCollection = collectionFromConstructor.merge(collectionFromImages)
print('mergedCollection: ', mergedCollection.getInfo())

# <font color="blue"> Select image Collection </font>

In [ ]:
VN=ee.FeatureCollection("users/havantuyen/VN_Map")
Map.centerObject(VN,10)
Map.addLayer(VN,{},"Vietnam")
Map

In [ ]:
# Load a Landsat 8 ImageCollection for a single path-row.
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterDate('2020-01-01', '2020-12-31').filterBounds(VN).\
filterMetadata("CLOUD_COVER","less_than",30)

In [ ]:
# MODIS temperature 
modis=ee.ImageCollection("MODIS/006/MOD11A1").filterDate('2020-01-01', '2020-12-31').filterBounds(VN).select("LST_Day_1km")

In [ ]:
# Get the number of images.
count = collection.size()
count.getInfo()

In [ ]:
# Get the date range of images in the collection.
range = collection.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
print('Date range: ', ee.Date(range.get('min')).getInfo(), ee.Date(range.get('max')).getInfo())

In [ ]:
# Better way of extracting start and end time
collection.get("date_range").getInfo()

In [ ]:
collection.reduceColumns(ee.Reducer.minMax(),["system:time_start"]).getInfo()

In [ ]:
# Change the date format.
range1 = collection.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
start_date = ee.Date(range1.get('min')).format("YYYY-MM-dd")
end_date = ee.Date(range1.get('max')).format("YYYY-MM-dd")
print('Date range: ', start_date.getInfo(), end_date.getInfo())

In [ ]:
# Get statistics for a property of the images in the collection.
sunStats = collection.aggregate_stats('RADIANCE_ADD_BAND_3')
print('Sun elevation statistics: ', sunStats.getInfo())

In [ ]:
# Get statistics for a property of the images in the collection.
clouds = collection.aggregate_array('CLOUD_COVER')
print('Cloud covers: ', clouds.getInfo())

In [ ]:
# Sort by a cloud cover property, get the least cloudy image.
image = ee.Image(collection.sort('CLOUD_COVER').first())
print('Least cloudy image: ', image.propertyNames().getInfo())
print('\nCloud cover: ', image.get("CLOUD_COVER").getInfo())

In [ ]:
# Limit the collection to the 10 most recent images.
recent = collection.sort('system:time_start', False).limit(10)
images = recent.aggregate_array('system:index').getInfo()
print("Recent images:")
for image in images:
    print(image)

# 1. Select Multiple Images

##### Filter or get `best` image from a ImageCollection

In [ ]:
# Filter date, bounds and sort by cloud cover
## Define the region of interest
point=ee.Geometry.Point(105.46,21.41)
## Filter by collection
collection=ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA").filterDate("2014-01-01","2015-01-01").\
filterBounds(point).sort("CLOUD_COVER")

In [ ]:
# Print out the number of images 
collection.size().getInfo()

In [ ]:
# Get the best image
image=collection.first()
Map.centerObject(image,10)
Map.addLayer(image,{},"HAAAA")
Map

In [ ]:
# Investigate the image property
image.getInfo()

In [ ]:
# Set property for image
date_code=ee.Date(image.get('system:time_start')).format("dd-MM-YYYY")

# date_code.getInfo()

addedProperty=image.set({"Date": date_code,"Name":"Ha Van Tuyen"})

print(addedProperty.get("Name").getInfo())

**Using `image.get('property')` to extract needed information from an image**

In [ ]:
# get property
image.get("RADIANCE_MULT_BAND_8").getInfo()

In [ ]:
# Visualizing the above image
vis={"bands":["B4","B3","B2"],"min":0.046, "max":0.389}
# Center the map
Map.centerObject(point,10)
# Plot the image
Map.addLayer(image, vis, "Image 2014")
Map

##### Write a function to select the best image by each year from 2015 to 2020

In [ ]:
def bestImage(year):
    start_year=ee.Date.fromYMD(year, 1,1)
    end_year=start_year.advance(1,"year")
    image=ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA").filterDate(start_year,end_year).\
filterBounds(point).sort("CLOUD_COVER").first()
    return image
years=ee.List.sequence(2014,2020)

myImage=years.map(bestImage)
# See the number of images
myImage.size().getInfo()

In [ ]:
collection=ee.ImageCollection(myImage)
# See cloud cover each image
for i in sorted(collection.aggregate_array("CLOUD_COVER").getInfo()):
    print("Cloud Cover: {:.3} %".format(i))

In [ ]:
# Visualize each image
for index in [0,1,2,3,4,5,6]:
    image=ee.Image(myImage.get(index))
    layerName="Image "+ str(years.getInfo()[index])
    Map.addLayer(image, vis, layerName, False)
Map

##### Writing a function to select monthly images 

In [6]:
years=range(2015,2017)

months=range(1,13)

import datetime as dt

for year in years:
    for month in months:
        start=ee.Date.fromYMD(year,month,1)
        end=start.advance(1, "month")
#         print("Starting month:",start.get("month").getInfo(),"-", end.get("month").getInfo())

## Converting GEE date to human datetime

In [ ]:
# Create a GEE date
date=ee.Date.fromYMD(2018,12,28)

date.getInfo()

In [ ]:
# We can convert a system time start to date time human
ee.Date(1545955200000).format("YYY,MM,dd").getInfo()

In [ ]:
# Using format to format to human datetime
date.format("YYYY-MM-dd").getInfo()

In [ ]:
# get all properties information
keys=image.getInfo()

for key in keys:
    if key=="properties":
        for i in keys[key]:
            print(i,":",keys[key][i])

In [ ]:
Map=emap.Map()
Map.add_basemap("Google Satellite")
Map
# Add Earth Engine dataset
image = ee.Image('USGS/SRTMGL1_003')

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Print the elevation of Mount Everest.
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = image.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

# Add Earth Engine layers to Map
Map.addLayer(image, vis_params, 'DEM')
Map.addLayer(xy, {'color': 'red'}, 'Mount Everest')

# Center the map based on an Earth Engine object or coordinates (longitude, latitude)
# Map.centerObject(xy, 4)
Map.setCenter(86.9250, 27.9881, 4)